In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import requests
import json
import uuid
import matplotlib.pyplot as plt
import base64
import io
import zipfile
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Import from supporting files
from main import load_files, find_unique_values, data_filter_setup, plotting_string_action, jv_plot_curve_best
from app_state import AppState
from data_processor import DataProcessor, JVDataExtractor, get_jv_data_for_analysis
sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_sample_description, get_all_JV, get_all_measurements_except_JV
from batch_selection import create_batch_selection

# Import from the new separate files
from auth_manager import AuthenticationManager, APIClient, STRINGS
from plotting_utils import WidgetFactory, LAYOUT
from error_handler import ErrorHandler

# Set the default renderer to 'notebook' for Jupyter
pio.renderers.default = 'notebook'
display(HTML("<script>document.title='NOMAD JV Analyzer'</script>"))

# --- Global Application State ---
batch_selection_container = widgets.Output()

# Initialize components
api_client = APIClient(STRINGS['SE_OASIS_URL'], STRINGS['API_ENDPOINT'])
app_state = AppState()
auth_manager = AuthenticationManager(api_client.get_base_url(), STRINGS['API_ENDPOINT'])

# URL Configuration
url_base = api_client.get_base_url()
url = api_client.get_api_url()

# --- Authentication Widgets ---
auth_method_selector = widgets.RadioButtons(
    options=['Username/Password', 'Token (from ENV)'],
    description='Auth Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin=LAYOUT['MARGIN_STANDARD'] + ' 0 0 0')
)

username_input = WidgetFactory.create_text_input(
    placeholder='Enter Username (e.g., email)',
    description='Username:'
)

password_input = WidgetFactory.create_text_input(
    placeholder='Enter Password',
    description='Password:',
    password=True
)

token_input = WidgetFactory.create_text_input(
    placeholder='Token will be read from ENV',
    description='Token:',
    width='wide',
    password=True
)
token_input.disabled = True

local_auth_box = widgets.VBox([username_input, password_input], layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0'))
token_auth_box = widgets.VBox([token_input], layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0', display='none'))

auth_button = WidgetFactory.create_button(
    description='Authenticate',
    button_style='info',
    tooltip='Authenticate using the selected method',
    icon='unlock',
    min_width=False
)
auth_button.layout.margin = LAYOUT['MARGIN_STANDARD'] + ' 0 0 0'

auth_status_label = widgets.Label(
    value=STRINGS['STATUS_NOT_AUTH'],
    layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0')
)

auth_action_box = widgets.VBox([auth_button, auth_status_label])

# --- Authentication Method Change Observer ---
def on_auth_method_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_method = change['new']
        if selected_method == 'Username/Password':
            local_auth_box.layout.display = 'flex'
            token_auth_box.layout.display = 'none'
        else:  # Token method
            local_auth_box.layout.display = 'none'
            token_auth_box.layout.display = 'none'
        
        auth_manager.clear_authentication()
        auth_manager._update_status(STRINGS['STATUS_AUTH_CHANGED'])

auth_method_selector.observe(on_auth_method_change, names='value')

# --- Authentication Button Handler ---
def update_auth_status(message, color=None):
    """Callback to update authentication status in UI"""
    auth_status_label.value = message
    if color:
        auth_status_label.style.text_color = color
    else:
        auth_status_label.style.text_color = None

# --- Authentication Helper Functions ---
def perform_username_password_auth():
    """Handle username/password authentication"""
    username = username_input.value
    password = password_input.value
    auth_manager.authenticate_with_credentials(username, password)
    password_input.value = ''  # Clear password after use

def perform_token_auth():
    """Handle token authentication"""
    auth_manager.authenticate_with_token()

def complete_authentication_process():
    """Complete the authentication process after successful login"""
    # Verify token and get user info
    user_info = auth_manager.verify_token()
    user_display = auth_manager.get_user_display_name()
    auth_manager._update_status(f'Status: Authenticated as {user_display} on SE Oasis.', 'green')
    
    # Enable the first tab if authentication is successful
    tabs.selected_index = 0  # Switch to the first tab after successful auth
    enable_tab(0)  # Enable the first tab
    
    # Close the settings box after successful authentication
    close_settings_box()
    
    # Initialize batch selection AFTER successful authentication
    init_batch_selection()

def on_auth_button_clicked(b):
    """Main authentication button handler"""
    auth_manager.clear_authentication()
    auth_manager.set_status_callback(update_auth_status)
    auth_manager._update_status(STRINGS['STATUS_AUTHENTICATING'], 'orange')

    try:
        if auth_method_selector.value == 'Username/Password':
            perform_username_password_auth()
        elif auth_method_selector.value == 'Token (from ENV)':
            perform_token_auth()
        
        complete_authentication_process()

    except Exception as e:
        ErrorHandler.handle_auth_error(e, auth_manager)

# Function to run authentication automatically on startup
def auto_authenticate():
    """Automatically run authentication when the notebook starts"""
    auth_manager.set_status_callback(update_auth_status)
    on_auth_button_clicked(None)

auth_button.on_click(on_auth_button_clicked)

# --- Combine Authentication Widgets ---
auth_box = widgets.VBox([
    auth_method_selector,
    local_auth_box,
    token_auth_box,
    auth_action_box
])

# --- Create collapsible settings box ---
settings_toggle_button = widgets.Button(
    description='▼ Connection Settings',
    button_style='',
    layout=widgets.Layout(width=LAYOUT['DROPDOWN_WIDTH'], margin='0 0 5px 0'),
    style={'font_weight': 'bold'}
)

settings_content = widgets.VBox([
    widgets.HTML("<p><strong>SE Oasis:</strong> https://nomad-hzb-se.de/nomad-oasis/api/v1</p>"),
    auth_box
], layout=widgets.Layout(padding=LAYOUT['PADDING_STANDARD'], margin='0 0 10px 0'))

# Initially visible settings content
general_settings_box = widgets.VBox([
    settings_toggle_button,
    settings_content
], layout=widgets.Layout(border=LAYOUT['BORDER_STANDARD'], padding=LAYOUT['PADDING_STANDARD'], margin='0 0 20px 0'))

# Function to toggle settings visibility
def toggle_settings(b):
    if settings_content.layout.display == 'none':
        # Show settings
        settings_content.layout.display = 'flex'
        settings_toggle_button.description = '▼ Connection Settings'
    else:
        # Hide settings
        settings_content.layout.display = 'none'
        settings_toggle_button.description = '▶ Connection Settings'

settings_toggle_button.on_click(toggle_settings)

# Function to close settings box (called after successful auth)
def close_settings_box():
    settings_content.layout.display = 'none'
    settings_toggle_button.description = '▶ Connection Settings'

# --- Initial Environment Detection ---
is_hub_environment = bool(os.environ.get('JUPYTERHUB_USER'))

if is_hub_environment:
    auth_method_selector.value = 'Token (from ENV)'
    local_auth_box.layout.display = 'none'
    token_auth_box.layout.display = 'none'
else:
    auth_method_selector.value = 'Username/Password'
    local_auth_box.layout.display = 'flex'
    token_auth_box.layout.display = 'none'

# --- Tab 1: Select Upload ---
# Re-using your existing widgets with adjusted layouts
load_status_output = WidgetFactory.create_output(min_height='standard', border=True)

# Tab 1 Handler Functions
def init_batch_selection():
    with batch_selection_container:
        clear_output(wait=True)
        if not auth_manager.is_authenticated():
            print(STRINGS['AUTH_REQUIRED'])
            return
        
        try:
            # Create the batch selection widget using the imported function
            batch_selection_widget = create_batch_selection(url, auth_manager.current_token, load_data_from_selection)
            display(batch_selection_widget)
        except Exception as e:
            ErrorHandler.log_error("initializing batch selection", e, batch_selection_container, show_traceback=True)

# --- Data Loading Helper Functions ---
def validate_data_loading_prerequisites(batch_selector):
    """Validate prerequisites for data loading"""
    if not auth_manager.is_authenticated():
        ErrorHandler.log_error(STRINGS['AUTH_REQUIRED'], None, load_status_output)
        return False
        
    if not batch_selector.value:
        ErrorHandler.log_error("Please select at least one batch to load", None, load_status_output)
        return False
    
    return True

def fetch_batch_data(batch_ids_list):
    """Fetch data for given batch IDs"""
    ErrorHandler.log_info(f"Loading data for batch IDs: {batch_ids_list}", load_status_output)
    
    sample_ids = get_ids_in_batch(url, token=auth_manager.current_token, batch_ids=batch_ids_list)
    identifiers = get_sample_description(url, token=auth_manager.current_token, sample_ids=sample_ids)            
    df_jvc, df_cur = get_jv_data_for_analysis(sample_ids, url, auth_manager.current_token, load_status_output)
    
    return sample_ids, identifiers, df_jvc, df_cur

def process_and_store_data(df_jvc, df_cur, identifiers):
    """Process and store the loaded data"""
    app_state.data["jvc"] = pd.concat([app_state.data.get("jvc", pd.DataFrame()), df_jvc], ignore_index=True)
    app_state.data["curves"] = pd.concat([app_state.data.get("curves", pd.DataFrame()), df_cur], ignore_index=True)
    
    # Process dataframes using utility functions
    app_state.data["jvc"] = DataProcessor.process_jv_dataframe(app_state.data["jvc"], identifiers)
    app_state.data["curves"] = DataProcessor.process_curves_dataframe(app_state.data["curves"])

def export_data_files(df_jvc, df_cur):
    """Export data to CSV files"""
    df_jvc.to_csv("export_jvc.csv")
    df_cur.to_csv("export_cur.csv")

def finalize_data_loading():
    """Finalize data loading process"""
    # Find unique values for variables
    app_state.unique_vals = find_unique_values(app_state.data["jvc"])
    
    # Enable and switch to the next tab
    enable_tab(1)
    tabs.selected_index = 1
    
    # Initialize variables tab with the loaded data
    make_variables_menu(app_state.unique_vals)

def load_data_from_selection(batch_selector):
    """Load data from selected batches"""
    app_state.clear_data()
    
    with load_status_output:
        clear_output(wait=True)
        ErrorHandler.log_info(STRINGS['LOADING_DATA'], load_status_output)
        
        if not validate_data_loading_prerequisites(batch_selector):
            return
        
        try:
            batch_ids_list = batch_selector.value
            sample_ids, identifiers, df_jvc, df_cur = fetch_batch_data(batch_ids_list)
            process_and_store_data(df_jvc, df_cur, identifiers)
            export_data_files(df_jvc, df_cur)
            
            ErrorHandler.log_success(STRINGS['DATA_LOADED_SUCCESS'], load_status_output)
            finalize_data_loading()
            
        except Exception as e:
            ErrorHandler.handle_data_loading_error(e, load_status_output)
            




# Organize Tab 1 Layout
select_upload_tab = widgets.VBox([
    widgets.HTML("<h3>Select Upload</h3>"),
    batch_selection_container,
    load_status_output
])

# --- Tab 2: Add Variable Names ---
default_variables = widgets.Dropdown(
    options=['all', 'Batch name', 'Variation'],
    value='Variation',
    description='Defaults:',
    disabled=False,
    style={'description_width': 'initial'}
)

dynamic_content = widgets.Output()
results_content = WidgetFactory.create_output(scrollable=True, border=False)
read_output = widgets.Output()

# Tab 2 Handler Functions
def create_widgets_table(elements_list):
    rows = []
    text_widgets = {}
    for item in elements_list:
        item_split = item.split("&")
        batch, variable = "", item
        if len(item_split) >= 2:
            batch, variable = item_split[0], "&".join(item_split[1:])
        default_value = ""
        if default_variables.value == "Batch name":
            default_value = batch if batch else "_".join(item.split("_")[:-1])
        if default_variables.value == "Variation":
            default_value = variable
        label = widgets.Label(value=variable)
        text_input = widgets.Text(value=default_value, placeholder='Variable e.g. 1000 rpm')
        row = widgets.HBox([label, text_input])
        rows.append(row)
        text_widgets[item] = text_input  
    return widgets.VBox(rows), text_widgets

def on_retrieve_clicked(text_widgets_dict):
    app_state.is_conditions = True
    conditions_dict = {}
    read_output.clear_output()
    for item, text_widget in text_widgets_dict.items():
        conditions_dict[item] = text_widget.value
    app_state.data['jvc']['condition'] = app_state.data['jvc']['identifier'].map(conditions_dict)
    
    with read_output:
        ErrorHandler.log_success(STRINGS['VARIABLES_LOADED'], read_output)
        
    # Show download button and measurements
    # FIX: Use the download_content Output widget instead of directly using download_area
    with download_content:
        clear_output(wait=True)
        display(download_button)
    
    # Automatically show measurements
    show_table(None)
    
    # Enable the next tab
    enable_tab(2)
    tabs.selected_index = 2

def brief_data_summary(df):
    global_mean_PCE = df['PCE(%)'].mean()
    global_std_PCE = df['PCE(%)'].std()
    max_PCE_row = df.loc[df['PCE(%)'].idxmax()]
    mean_std_PCE_per_sample = df.groupby(['batch', 'sample'])['PCE(%)'].agg(['mean', 'std'])
    highest_mean_PCE_sample = mean_std_PCE_per_sample.idxmax()['mean']
    lowest_mean_PCE_sample = mean_std_PCE_per_sample.idxmin()['mean']
    highest_PCE_per_sample = df.loc[df.groupby(['sample'])['PCE(%)'].idxmax(), ['sample', 'cell', 'PCE(%)']]
    
    markdown_output = f"""
### Summary Statistics

**Global mean PCE(%)**: {global_mean_PCE:.2f} ± {global_std_PCE:.2f}%


|   | Sample | Mean PCE(%) | Std PCE(%) |
|---|--------|-------------|------------|
| Best sample | {highest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'std']:.2f}% |
| Worst sample | {lowest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'std']:.2f}% |

#### Highest PCE(%) per Sample

| | Sample | Cell | PCE(%) |
|-|--------|------|--------|
| Best Overall | {max_PCE_row['sample']} | {max_PCE_row['cell']} | {max_PCE_row['PCE(%)']:.2f}% |
"""
    for sample, row in highest_PCE_per_sample.set_index('sample').iterrows():
        markdown_output += f"| | {sample} | {row['cell']} | {row['PCE(%)']:.2f}% |\n"
        
    return markdown_output

# --- Variables Menu Helper Functions ---
def generate_variables_markdown(unique_vals):
    """Generate markdown content for variables menu"""
    return f"""
# Add variable names
There are {len(unique_vals)} samples found.
If you tested specific variables or conditions for each sample, please write them down below.
"""

def create_variables_widgets(unique_vals):
    """Create widgets for variable input"""
    widgets_table, text_widgets_dict = create_widgets_table(unique_vals)
    retrieve_button = widgets.Button(
        description="Confirm variables",
        button_style='success',
        layout=widgets.Layout(min_width=LAYOUT['BUTTON_MIN_WIDTH'])
    )
    retrieve_button.on_click(lambda b: on_retrieve_clicked(text_widgets_dict))
    return widgets_table, text_widgets_dict, retrieve_button

def display_variables_interface(unique_vals):
    """Display the complete variables interface"""
    variables_markdown = generate_variables_markdown(unique_vals)
    results_markdown = brief_data_summary(app_state.data['jvc'])
    
    with dynamic_content:
        clear_output(wait=True)
        display(Markdown(variables_markdown))
        display(default_variables)
        
        widgets_table, text_widgets_dict, retrieve_button = create_variables_widgets(unique_vals)
        information_group = widgets.HBox([widgets_table, results_content])
        button_group = widgets.HBox([retrieve_button, read_output])
        
        display(information_group)
        display(button_group)
    
    return results_markdown

def initialize_results_display(results_markdown):
    """Initialize the results display area"""
    with results_content:
        results_content.clear_output()
        display(Markdown(results_markdown))
    
    with read_output:
        read_output.clear_output()
        print(STRINGS['VARIABLES_NOT_LOADED'])

def make_variables_menu(unique_vals):
    """Create and display the variables menu"""
    results_markdown = display_variables_interface(unique_vals)
    initialize_results_display(results_markdown)
        
def on_change_default_variables(b):
    dynamic_content.clear_output()
    make_variables_menu(app_state.unique_vals)

default_variables.observe(on_change_default_variables, names=['value'])

# Quick download section
download_button = widgets.Button(
    description='Download JV',
    button_style='info',
    layout=widgets.Layout(min_width=LAYOUT['BUTTON_MIN_WIDTH'])
)

download_content = widgets.Output()
download_container = widgets.VBox([download_content])

def trigger_download(text, filename, kind='text/json'):
    content_b64 = base64.b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_content:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

def download_jv_data(e=None):
    jvc = io.StringIO()
    curves = io.StringIO()
    app_state.data["jvc"].to_csv(jvc)
    app_state.data["curves"].to_csv(curves)
    trigger_download(jvc.getvalue(), 'jvc.csv', kind='text/plain')
    trigger_download(curves.getvalue(), 'curves.csv', kind='text/plain')

download_button.on_click(download_jv_data)

# Show measurements section
show_other_measurements = widgets.Output()
measurements_container = widgets.VBox([show_other_measurements])

def show_table(e=None):
    if not auth_manager.is_authenticated():
        with show_other_measurements:
            clear_output(wait=True)
            print("Please authenticate and load data first.")
        return
        
    with show_other_measurements:
        clear_output(wait=True)
        print("Loading measurements data...")
        
        try:
            # Get the selected batch IDs from the loaded data
            batch_ids_value = list(app_state.data['jvc']['batch'].unique()) if 'jvc' in app_state.data and 'batch' in app_state.data['jvc'] else []
            
            if not batch_ids_value:
                print("No batch IDs found in loaded data.")
                return
                
            sample_ids = get_ids_in_batch(url, auth_manager.current_token, batch_ids_value)
            measurements_data = get_all_measurements_except_JV(url, auth_manager.current_token, sample_ids)

            df = pd.DataFrame()
            for key, value in measurements_data.items():
                df[key] = pd.Series([DataProcessor.create_measurement_link(r[1]) for r in value])
            
            if df.empty:
                print("No additional measurements found.")
            else:
                display(HTML("<h3>Additional Measurements</h3>"))
                display(HTML(df.to_html(escape=False)))
        
        except Exception as e:
            ErrorHandler.log_error("displaying measurements", e, show_other_measurements)

# Organize Tab 2 Layout
add_variables_tab = widgets.VBox([
    widgets.HTML("<h3>Add Variable Names</h3>"),
    dynamic_content,
    widgets.HTML("<h3>Download Data</h3>"),
    download_container,
    widgets.HTML("<h3>Other Measurements</h3>"),
    measurements_container
])

# --- Tab 3: Select Filters ---
filter_presets = {
    "Default": [("Jsc(mA/cm2)", ">=", "-30"), ("Voc(V)", "<=", "2"), ("FF(%)", ">=", "24"), ("FF(%)", "<=", "89"), ("PCE(%)", "<=", "40")],
    "Preset 2": [("FF(%)", "<", "15"), ("PCE(%)", ">=", "10")]
}

# Create filter widgets
preset_dropdown_filter = WidgetFactory.create_dropdown(
    options=list(filter_presets.keys()),
    description='Filters'
)

direction_radio = WidgetFactory.create_radio_buttons(
    options=['Both', 'Reverse', 'Forward'],
    description='Direction:',
    value='Both'
)

add_button_filter = WidgetFactory.create_button("Add Filter", button_style='primary', min_width=True)
remove_button_filter = WidgetFactory.create_button("Remove Filter", button_style='danger', min_width=True)
apply_preset_filter_button = WidgetFactory.create_button("Load Preset", button_style='info', min_width=True)
apply_filter_button = WidgetFactory.create_button("Apply Filter", button_style='success')
confirmation_filter = widgets.Output()
main_output_content = WidgetFactory.create_output(min_height='large', scrollable=True, border=True)

def create_widget_group_filter(include_headers=False):
    if include_headers:
        headers = widgets.HBox([
            widgets.Label('Variable', layout=widgets.Layout(width=LAYOUT['LABEL_WIDTH'])),
            widgets.Label('Operator', layout=widgets.Layout(width=LAYOUT['LABEL_WIDTH'])),
            widgets.Label('Value', layout=widgets.Layout(width='auto'))
        ])
        row = WidgetFactory.create_filter_row()
        return widgets.VBox([headers, row])
    else:
        return WidgetFactory.create_filter_row()

widget_groups_filter = [create_widget_group_filter(include_headers=False)]
groups_container_filter = widgets.VBox(widget_groups_filter)

# Tab 3 Handler Functions
def add_widget_group_filter(b):
    widget_groups_filter.append(create_widget_group_filter())
    groups_container_filter.children = widget_groups_filter

def remove_widget_group_filter(b):
    if len(widget_groups_filter) > 1:
        widget_groups_filter.pop()
        groups_container_filter.children = widget_groups_filter

def apply_preset_filter(b):
    selected_preset = preset_dropdown_filter.value
    widget_groups_filter.clear()
    if selected_preset in filter_presets:
        for variable, operator, value in filter_presets[selected_preset]:
            group = create_widget_group_filter()
            group.children[0].value = variable
            group.children[1].value = operator
            group.children[2].value = value
            widget_groups_filter.append(group)
    else:
        widget_groups_filter.append(create_widget_group_filter(include_headers=False))
    groups_container_filter.children = widget_groups_filter

# --- Filtering Helper Functions ---
def extract_filter_values():
    """Extract filter values from UI widgets"""
    filter_values = []
    for group in widget_groups_filter:
        variable = group.children[0].value
        operator = group.children[1].value
        value = group.children[2].value
        filter_values.append((variable, operator, value))
    return filter_values

def apply_numeric_filters(filter_values):
    """Apply numeric filters to the data"""
    app_state.data['filtered'], app_state.data['junk'], app_state.filter_vals = data_filter_setup(
        app_state.data['jvc'], filter_values
    )

def apply_direction_filter(direction_value):
    """Apply direction filter if specified"""
    if direction_value != 'Both':
        app_state.data['filtered'] = app_state.data['filtered'][
            app_state.data['filtered']['direction'] == direction_value
        ]
        direction_mask = app_state.data['jvc']['direction'] != direction_value
        app_state.data['junk'] = pd.concat([
            app_state.data['junk'], 
            app_state.data['jvc'][direction_mask & ~app_state.data['jvc'].index.isin(app_state.data['junk'].index)]
        ])

def display_filtering_results():
    """Display the results of filtering"""
    retained_count = len(app_state.data['filtered'])
    filtered_count = len(app_state.data['junk'])
    
    ErrorHandler.log_success(
        f"Filtering complete! Retained {retained_count} records, filtered out {filtered_count} records.",
        main_output_content
    )
    ErrorHandler.log_info("You can proceed to the next tab to create plots.", main_output_content)

def apply_filtering(b):
    """Apply all filters to the data"""
    main_output_content.clear_output()
    confirmation_filter.clear_output()
    
    with main_output_content:
        try:
            filter_values = extract_filter_values()
            direction_value = direction_radio.value
            
            apply_numeric_filters(filter_values)
            apply_direction_filter(direction_value)
            display_filtering_results()
            
            # Enable the next tab
            enable_tab(3)
            
        except Exception as e:
            ErrorHandler.log_error("applying filters", e, main_output_content)

# Connect handlers to buttons
add_button_filter.on_click(add_widget_group_filter)
remove_button_filter.on_click(remove_widget_group_filter)
apply_preset_filter_button.on_click(apply_preset_filter)
apply_filter_button.on_click(apply_filtering)

# Initialize with preset
apply_preset_filter(None)

# Filter tab layout components
direction_label = widgets.HTML(value="<b>Filter by Cell Direction:</b>")
direction_container = widgets.VBox([direction_label, direction_radio])

filter_conditions_label = widgets.HTML(value="<b>Filter Conditions:</b>")
filter_conditions_container = widgets.VBox([filter_conditions_label, groups_container_filter])

confirm_group_filter = widgets.HBox([apply_filter_button, confirmation_filter])
controls_filter = widgets.VBox([
    add_button_filter, 
    remove_button_filter, 
    preset_dropdown_filter, 
    apply_preset_filter_button,
    confirm_group_filter
])

# Organize Tab 3 Layout
filter_tab_layout = widgets.GridspecLayout(1, 3)
filter_tab_layout[0, 0] = controls_filter
filter_tab_layout[0, 1] = widgets.VBox([direction_container, filter_conditions_container])
filter_tab_layout[0, 2] = main_output_content

select_filters_tab = widgets.VBox([
    widgets.HTML("<h3>Select Filters</h3>"),
    widgets.HTML("<p>Using the dropdowns below, select filters for the data you want to keep, not remove.</p>"),
    filter_tab_layout
])

# --- Tab 4: Select Plots ---
plot_presets = {
    "Default": [("Boxplot", "PCE", "by Variable"), ("Boxplot", "Voc", "by Variable"), ("Boxplot", "Jsc", "by Variable"),
                ("Boxplot", "FF", "by Variable"), ("JV Curve", "Best device only", "All annotations")],
    "Preset 2": [("Boxplot", "Voc", "by Cell"), ("Histogram", "Voc", ""), ("JV Curve", "Best device only", "All annotations")],
    "Status Analysis": [("Boxplot", "PCE", "by Status"), ("Paired Boxplot", "PCE", "by Status")]  # ADD THIS NEW PRESET
}

# Create plot widgets
load_preset_plot_button = WidgetFactory.create_button("Load Preset", button_style='info', min_width=False)

preset_dropdown_plot = WidgetFactory.create_dropdown(
    options=list(plot_presets.keys()),
    description='Presets'
)

add_button_plot = WidgetFactory.create_button("Add Plot Type", button_style='primary', min_width=False)

remove_button_plot = WidgetFactory.create_button("Remove Plot Type", button_style='danger', min_width=False)

plot_selection_button = WidgetFactory.create_button("Plot Selection", button_style='success')

plotted_content = widgets.Output()

# Function to update additional options based on plot type selection
def update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown):
    plot_type = plot_type_dropdown.value
    if plot_type == 'Boxplot':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction', 'by Status']
    elif plot_type == 'Paired Boxplot':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Status']
    elif plot_type == 'Boxplot (omitted)':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction', 'by Status']
    elif plot_type == 'Histogram':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['']  # Histogram does not use the second option
    elif plot_type == 'JV Curve':
        option1_dropdown.options = ['All cells', 'Only working cells', 'Only not working cells', 'Best device only', 'Separated by cell', 'Separated by substrate']
        option2_dropdown.options = ['']  # Default to empty, will be updated by observer
    else:
        option1_dropdown.options = []
        option2_dropdown.options = []

# Function to create a new plot type group
def create_plot_type_group():
    plot_type_dropdown = widgets.Dropdown(
        options=['Boxplot', 'Paired Boxplot', 'Boxplot (omitted)', 'Histogram', 'JV Curve'], 
        description='Plot Type:', 
        layout=widgets.Layout(width=LAYOUT['DROPDOWN_WIDE'])
    )
    option1_dropdown = widgets.Dropdown(
        description='Option 1:',
        layout=widgets.Layout(width=LAYOUT['DROPDOWN_EXTRA_WIDE'])
    )
    option2_dropdown = widgets.Dropdown(
        description='Option 2:',
        layout=widgets.Layout(width=LAYOUT['DROPDOWN_EXTRA_WIDE'])
    )
    
    # Function to update option2 when option1 changes for JV Curve
    def update_option2_for_jv(change):
        if plot_type_dropdown.value == 'JV Curve':
            if change['new'] == 'Best device only':
                option2_dropdown.options = ['All annotations', 'No grid', 'No performance', 'No grid and no performance']
                if 'All annotations' in option2_dropdown.options:
                    option2_dropdown.value = 'All annotations'
            else:
                option2_dropdown.options = ['']
                option2_dropdown.value = ''
    
    # Initial update for default selection
    update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown)
    
    # Observer for plot type changes
    plot_type_dropdown.observe(
        lambda change: update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown), 
        names='value'
    )
    
    # Observer for option1 changes in JV Curve
    option1_dropdown.observe(update_option2_for_jv, names='value')
    
    return widgets.HBox([plot_type_dropdown, option1_dropdown, option2_dropdown])

plot_type_groups = [create_plot_type_group()]
groups_container = widgets.VBox(plot_type_groups)

# Plot tab handler functions
def add_plot_type_group(b):
    new_group = create_plot_type_group()
    plot_type_groups.append(new_group)
    groups_container.children = tuple(plot_type_groups)

def remove_plot_type_group(b):
    if len(plot_type_groups) > 1:
        plot_type_groups.pop()
        groups_container.children = tuple(plot_type_groups)

def load_preset_plot(b):
    selected_preset = preset_dropdown_plot.value
    plot_type_groups.clear()
    if selected_preset in plot_presets:
        for plot_type, option1, option2 in plot_presets[selected_preset]:
            new_group = create_plot_type_group()
            new_group.children[0].value = plot_type  # Set plot type
            # Update options first, then set values
            update_additional_options(new_group.children[0], new_group.children[1], new_group.children[2])
            new_group.children[1].value = option1    # Set option 1
            
            # Special handling for JV Curve to update option2 options
            if plot_type == 'JV Curve' and option1 == 'Best device only':
                new_group.children[2].options = ['All annotations', 'No grid', 'No performance', 'No grid and no performance']
            
            # Only set option2 if it's in the available options
            if option2 in new_group.children[2].options:
                new_group.children[2].value = option2
            elif new_group.children[2].options:  # If there are options, use the first one
                new_group.children[2].value = new_group.children[2].options[0]
                
            plot_type_groups.append(new_group)
    else:
        plot_type_groups.append(create_plot_type_group())
    groups_container.children = tuple(plot_type_groups)

# --- Plotting Helper Functions ---
def extract_plot_configurations():
    """Extract plot configurations from UI widgets"""
    extracted_contents = []
    for group in plot_type_groups:
        plot_type_widget, option1_widget, option2_widget = group.children[0], group.children[1], group.children[2]
        plot_type = plot_type_widget.value
        option1 = option1_widget.value
        option2 = option2_widget.value
        extracted_contents.append((plot_type, option1, option2))
    return extracted_contents

def validate_plotting_prerequisites():
    """Validate prerequisites for plotting"""
    if 'filtered' not in app_state.data:
        ErrorHandler.log_error("Please apply filters first in the 'Select Filters' tab", None, plotted_content)
        return False
    return True

def save_full_data_frame(data):
    """Create a workbook with JV data for Excel export"""
    import openpyxl
    from openpyxl.utils.dataframe import dataframe_to_rows
    
    wb = openpyxl.Workbook()
    wb.remove(wb.active)  # Remove the default sheet
    
    # Write the DataFrame to a sheet named 'All_data'
    ws = wb.create_sheet(title='All_data')
    for r in dataframe_to_rows(data, index=True, header=True):
        ws.append(r)
    
    return wb

def prepare_plot_data():
    """Prepare data for plotting"""
    wb = save_full_data_frame(app_state.data['jvc'])
    data_plot = app_state.data['filtered'], app_state.data["jvc"], app_state.data["curves"]
    p_rel = os.getcwd()
    supp_plot = app_state.data['junk'], app_state.filter_vals, app_state.is_conditions, p_rel, app_state.data["curves"]["sample"].unique().tolist()
    return wb, data_plot, supp_plot

def handle_customizable_jv_plot(extracted_contents):
    """Handle customizable JV plot if requested"""
    for plot_type, option1, option2 in extracted_contents:
        if plot_type == 'JV Curve' and option1 == 'Best device (customizable)':
            jv_interface, update_func = create_jv_plot_controls()
            
            with plotted_content:
                clear_output(wait=True)
                display(widgets.HTML("<h3>JV Curve - Best Device (Customizable)</h3>"))
                display(jv_interface)
                update_func()  # Initial plot creation
            
            enable_tab(4)
            return True
    return False

def create_plotly_figure_html(fig, i):
    """Create HTML for plotly figure with responsive features - enhanced for full resizing"""
    # Update the figure layout to be responsive
    fig.update_layout(
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50, autoexpand=True),
        width=None,
        height=None
    )
    
    # Convert to HTML and modify it to be truly responsive
    fig_html = fig.to_html(include_plotlyjs='cdn', div_id=f"plot_{i}")
    
    # Inject responsive CSS and JavaScript directly into the HTML
    responsive_additions = f"""
    <style>
        body, html {{
            margin: 0;
            padding: 0;
            width: 100%;
            height: 100%;
            overflow: hidden;
        }}
        .plotly-graph-div {{
            width: 100% !important;
            height: 100% !important;
            box-sizing: border-box;
        }}
        #plot_{i} {{
            width: 100% !important;
            height: 100% !important;
            box-sizing: border-box;
            overflow: hidden;
        }}
    </style>
    <script>
        console.log("Hello world!");
        // Enhanced message handling for full resize support
        window.addEventListener('message', function(e) {{
            console.log(e);
            console.log("1Hello world!");
            if (e.data && e.data.type === 'resize_{i}') {{
                console.log(e);
                var plotDiv = document.getElementById('plot_{i}');
                if (plotDiv && typeof Plotly !== 'undefined') {{
                    var newWidth = Math.max(300, e.data.width - 20);
                    var newHeight = Math.max(200, e.data.height - 20);
                    
                    // Force container sizing first
                    plotDiv.style.width = newWidth + 'px';
                    plotDiv.style.height = newHeight + 'px';
                    
                    // Use newPlot for complete recreation with proper background sizing
                    var currentData = plotDiv.data;
                    var currentLayout = plotDiv.layout;
                    var currentConfig = plotDiv.config || {{}};
                    
                    // Update layout with new dimensions
                    currentLayout.width = newWidth;
                    currentLayout.height = newHeight;
                    currentLayout.autosize = false;
                    
                    // Recreate the plot entirely
                    Plotly.newPlot(plotDiv, currentData, currentLayout, currentConfig);
                }}
            }}
        }});
        
        // Auto-resize when window changes
        window.addEventListener('resize', function() {{
            var plotDiv = document.getElementById('plot_{i}');
            if (plotDiv && typeof Plotly !== 'undefined') {{
                setTimeout(function() {{
                    Plotly.Plots.resize(plotDiv);
                }}, 100);
            }}
        }});
        
        // Initial setup
        document.addEventListener('DOMContentLoaded', function() {{
            var plotDiv = document.getElementById('plot_{i}');
            if (plotDiv && typeof Plotly !== 'undefined') {{
                // Ensure the plot fills the container initially
                setTimeout(function() {{
                    Plotly.Plots.resize(plotDiv);
                }}, 100);
            }}
        }});
    </script>
    """
    
    # Insert the responsive code before the closing </body> tag
    return fig_html.replace('</body>', responsive_additions + '</body>')


def create_resizable_container_html(fig_html, name, i):
    """Create resizable container HTML with working resize functionality"""
    return f"""
    <div style="margin: 20px 0;">
        <h3 style="margin: 0 0 10px 0; font-weight: bold;">{name}</h3>
        <div id="container_{i}" style="
            min-width: 400px; 
            min-height: 300px; 
            width: 800px; 
            height: 500px;
            border: 2px solid #ddd;
            border-radius: 8px;
            position: relative;
            background: white;
            resize: both;
            overflow: auto;
        ">
            <iframe id="iframe_{i}" srcdoc='{fig_html.replace("'", "&apos;")}' 
                    style="
                        width: 100%; 
                        height: 100%; 
                        border: none; 
                        display: block;
                        background: white;
                        pointer-events: none;
                    ">
            </iframe>
        </div>
    </div>
    <script>
    console.log("5Hello world!");
    (function() {{
        console.log("4Hello world!");
        var container = document.getElementById('container_{i}');
        var iframe = document.getElementById('iframe_{i}');

        if (!container || !iframe) return;
        
        function sendResize() {{
            console.log("2Hello world!");
            if (iframe.contentWindow) {{
                try {{
                    iframe.contentWindow.postMessage({{
                        'type': 'resize_{i}', 
                        'width': container.offsetWidth, 
                        'height': container.offsetHeight
                    }}, '*');
                }} catch(e) {{
                    // Ignore messaging errors
                }}
            }}
        }}
        
        iframe.onload = function() {{
            setTimeout(sendResize, 200);
        }};
        
        // Use ResizeObserver to detect when container is resized
        if (window.ResizeObserver) {{
            var resizeObserver = new ResizeObserver(function(entries) {{
                setTimeout(sendResize, 50);
            }});
            resizeObserver.observe(container);
        }} else {{
            // Fallback: poll for size changes
            var lastWidth = container.offsetWidth;
            var lastHeight = container.offsetHeight;
            
            setInterval(function() {{
                var currentWidth = container.offsetWidth;
                var currentHeight = container.offsetHeight;
                
                if (currentWidth !== lastWidth || currentHeight !== lastHeight) {{
                    lastWidth = currentWidth;
                    lastHeight = currentHeight;
                    sendResize();
                }}
            }}, 200);
        }}
        
        // Initial resize
        setTimeout(sendResize, 100);
    }})();
    </script>
    """

def display_plotly_figure_enhanced(fig, name, i):
    """Enhanced display function for plotly figures with full resizing support"""
    try:
        # Update figure to be fully responsive
        fig.update_layout(
            autosize=True,
            margin=dict(autoexpand=True)
        )
        
        fig_html = create_plotly_figure_html(fig, i)
        resizable_html = create_resizable_container_html(fig_html, name, i)
        display(widgets.HTML(resizable_html))
        
    except Exception as e:
        print(f"Error displaying plot {name}: {e}")
        # Fallback to simple display
        display(widgets.HTML(f"<h3>{name}</h3>"))
        display(fig)

def display_matplotlib_figure_enhanced(fig, name, i):
    """Enhanced display function for matplotlib figures with resizing"""
    import io
    import base64
    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=150, bbox_inches='tight')
    buf.seek(0)
    img_b64 = base64.b64encode(buf.getvalue()).decode()
    
    resizable_html = f"""
    <div style="margin: 20px 0;">
        <h3 style="margin: 0 0 10px 0; font-weight: bold;">{name}</h3>
        <div style="resize: both; 
                   overflow: hidden; 
                   min-width: 400px; 
                   min-height: 300px; 
                   width: 800px; 
                   height: 500px;
                   border: 2px solid #ddd;
                   border-radius: 8px;
                   position: relative;
                   display: flex;
                   align-items: center;
                   justify-content: center;
                   background: white;">
            <img src="data:image/png;base64,{img_b64}" 
                 style="max-width: 100%; max-height: 100%; object-fit: contain;" />
            <div style="position: absolute; 
                       bottom: 2px; 
                       right: 2px; 
                       width: 15px; 
                       height: 15px; 
                       background: linear-gradient(-45deg, transparent 0%, transparent 40%, #666 40%, #666 60%, transparent 60%);
                       cursor: nw-resize;
                       pointer-events: none;
                       border-radius: 0 0 6px 0;">
            </div>
        </div>
    </div>
    """
    
    display(widgets.HTML(resizable_html))

def display_single_plot(fig, name, i):
    """Display a single plot (plotly or matplotlib) with enhanced resizing"""
    display_single_plot_enhanced(fig, name, i)

def display_single_plot_enhanced(fig, name, i):
    """Enhanced version of display_single_plot with proper resizing"""
    print(f"\nPlot {i+1}: {name}")
    
    try:
        if hasattr(fig, 'data'):  # Check if it's a Plotly figure
            display_plotly_figure_enhanced(fig, name, i)
        else:
            # For matplotlib figures, use the enhanced approach
            display_matplotlib_figure_enhanced(fig, name, i)
    except Exception as e:
        print(f"Error displaying plot {i+1} ({name}): {e}")
        # Try alternative display method
        try:
            display(widgets.HTML(f"<h3>{name}</h3>"))
            display(fig)
        except Exception as fallback_error:
            print(f"Could not display plot {name}: {fallback_error}")

def display_all_plots(figs, names):
    """Display all generated plots"""
    ErrorHandler.log_info(f"Displaying {len(figs)} plots...", plotted_content)
    
    for i, (fig, name) in enumerate(zip(figs, names)):
        display_single_plot(fig, name, i)
    
    ErrorHandler.log_info("\nPlots created successfully! Proceed to the next tab to save your results.", plotted_content)
    enable_tab(4)

def plot_selected_options(b):
    """Main plotting function"""
    with plotted_content:
        clear_output(wait=True)
        ErrorHandler.log_info("Creating plots, please wait...", plotted_content)
    
    try:
        if not validate_plotting_prerequisites():
            return
        
        extracted_contents = extract_plot_configurations()
        
        # Check for customizable plots first
        if handle_customizable_jv_plot(extracted_contents):
            return
        
        # Prepare data for plotting
        wb, data_plot, supp_plot = prepare_plot_data()
        
        figs, names = plotting_string_action(extracted_contents, data_plot, supp_plot, True)
        workbook = wb  # Use the wb you created earlier
        app_state.update_plot_data(figs, names, workbook)
        
        # Display all plots
        with plotted_content:
            clear_output(wait=True)
            display_all_plots(figs, names)
            
    except Exception as e:
        with plotted_content:
            clear_output(wait=True)
            ErrorHandler.handle_plot_error(e, plotted_content)
            ErrorHandler.log_info("Please make sure you have applied filters in the previous tab.", plotted_content)

plot_selection_button.on_click(plot_selected_options)

# Connect handlers to buttons
add_button_plot.on_click(add_plot_type_group)
remove_button_plot.on_click(remove_plot_type_group)
load_preset_plot_button.on_click(load_preset_plot)
plot_selection_button.on_click(plot_selected_options)

# Initialize with default preset
load_preset_plot(None)

# Organize Tab 4 Layout
controls_plot = widgets.VBox([
    add_button_plot, 
    remove_button_plot, 
    preset_dropdown_plot,
    load_preset_plot_button,
    plot_selection_button
])

select_plots_tab = widgets.VBox([
    widgets.HTML("<h3>Select Plots</h3>"),
    widgets.HTML("<p>Using the dropdowns below, select the plots you want to create.</p>"),
    widgets.HBox([controls_plot, groups_container]),
    plotted_content
])

# --- Tab 5: Save Plots and Data ---
save_plots_button = WidgetFactory.create_button(
    'Save All Plots',
    button_style='primary',
    tooltip='Click to save all plots',
    min_width=False
)

save_data_button = WidgetFactory.create_button(
    'Save Data',
    button_style='info',
    tooltip='Click to save the collected raw data',
    min_width=False
)

save_all_button = WidgetFactory.create_button(
    'Save Data & Plots',
    button_style='success',
    tooltip='Click to save the collected raw data and all plots',
    min_width=False
)

download_save_output = widgets.Output()

# Handler functions for saving plots and data
def on_save_plots_button_clicked(b):
    download_save_output.clear_output()

    if not app_state.global_plot_data.get('figs'):
        with download_save_output:
            print("No plots have been created yet. Please create plots in the previous tab.")
        return

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for i, (fig, name) in enumerate(zip(app_state.global_plot_data['figs'], app_state.global_plot_data['names'])):
            try:
                # Save as HTML
                html_str = fig.to_html(include_plotlyjs='cdn')
                zip_file.writestr(f"{name}", html_str)
                
                # Try to save as PNG if kaleido is installed
                try:
                    import plotly.io as pio
                    img_bytes = pio.to_image(fig, format='png')
                    zip_file.writestr(f"{name.replace('.html', '.png')}", img_bytes)
                except Exception as img_err:
                    pass
            except Exception as e:
                ErrorHandler.log_error(f"saving {name}", e, download_save_output)

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'plots.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all plots</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

def on_save_data_button_clicked(b):
    download_save_output.clear_output()

    if not app_state.global_plot_data.get('workbook'):
        with download_save_output:
            print("No data has been processed yet. Please create plots in the previous tab.")
        return

    # Save Excel file to memory
    excel_buffer = io.BytesIO()
    app_state.global_plot_data['workbook'].save(excel_buffer)
    excel_buffer.seek(0)

    # Encode to base64
    b64 = base64.b64encode(excel_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}';
    link.download = 'collected_data.xlsx';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download data</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

def on_save_all_button_clicked(b):
    download_save_output.clear_output()

    if not app_state.global_plot_data.get('figs') or not app_state.global_plot_data.get('workbook'):
        with download_save_output:
            print("No plots or data have been created yet. Please create plots in the previous tab.")
        return

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for i, (fig, name) in enumerate(zip(app_state.global_plot_data['figs'], app_state.global_plot_data['names'])):
            try:
                # Save as HTML
                html_str = fig.to_html(include_plotlyjs='cdn')
                zip_file.writestr(f"{name}", html_str)
                
                # Try to save as PNG if kaleido is installed
                try:
                    import plotly.io as pio
                    img_bytes = pio.to_image(fig, format='png')
                    zip_file.writestr(f"{name.replace('.html', '.png')}", img_bytes)
                except Exception as img_err:
                    pass
            except Exception as e:
                ErrorHandler.log_error(f"saving {name}", e, download_save_output)

        # Add Excel file to zip
        excel_buffer = io.BytesIO()
        app_state.global_plot_data['workbook'].save(excel_buffer)
        excel_buffer.seek(0)
        zip_file.writestr("collected_data.xlsx", excel_buffer.getvalue())

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'results.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_save_output:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all files</button>"))
        print("Download initiated. If the download doesn't start automatically, click the button above.")

# Attach the event handlers to the buttons
save_plots_button.on_click(on_save_plots_button_clicked)
save_data_button.on_click(on_save_data_button_clicked)
save_all_button.on_click(on_save_all_button_clicked)

# Organize Tab 5 Layout
save_plots_and_data_tab = widgets.VBox([
    widgets.HTML("<h3>Save Plots and Data</h3>"),
    widgets.HBox([save_plots_button, save_data_button, save_all_button]),
    download_save_output
])

# --- Tab System Controller ---
# Dynamic tab enabling/disabling
tab_labels = ['Select Upload', 'Add Variable Names', 'Select Filters', 'Select Plots', 'Save Results']

# Create the tab widget
tabs = widgets.Tab()
tabs.children = [
    select_upload_tab, 
    add_variables_tab,
    select_filters_tab,
    select_plots_tab,
    save_plots_and_data_tab
]

# Set tab titles
for i, title in enumerate(tab_labels):
    tabs.set_title(i, title)

# Function to control tab enabling/disabling
def enable_tab(tab_index):
    """Enables the specified tab and updates styles"""
    app_state.enable_tab(tab_index)

# --- Main Dashboard Layout ---
app_layout = widgets.Layout(
    max_width=LAYOUT['CONTAINER_MAX_WIDTH'],
    margin="0 auto",
    padding=LAYOUT['PADDING_LARGE']
)

dashboard_layout = widgets.VBox([
    widgets.HTML("<h1>JV Analysis Dashboard</h1>"),
    general_settings_box,
    tabs
], layout=app_layout)

# Display the dashboard
display(dashboard_layout)
auto_authenticate()

# --- Start with authentication tab ---
# Initial state: only authentication is enabled
enable_tab(0)  # Enable the first tab (Upload)
